In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [10]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])

In [20]:
CRIM_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
NOX_IDX = 3
RM_IDX = 4
DIS_IDX = 5
RAD_IDX = 6
TAX_IDX = 7
PTRATIO_IDX = 8
B_IDX = 9
LSTAT_IDX = 10

property_stats = features.mean().values.reshape(1, 11)

property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [27]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# MSE and RMSE
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [45]:
def get_log_estimate(nr_rooms,
                    students_per_classroom,
                    next_to_river=False,
                    high_confidence=True):
    
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
         property_stats[0][CHAS_IDX] = 1
    else:
         property_stats[0][CHAS_IDX] = 0
              
    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc range
    if high_confidence:
        upper_bound = log_estimate + 2 * RMSE
        lower_bound = log_estimate - 2 * RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 65
    
    return log_estimate, upper_bound, lower_bound, interval

In [46]:
get_log_estimate(3, 20, next_to_river=True)

(2.776758191480399, 3.1517824618746597, 2.4017339210861386, 95)

In [59]:
# Converting the log price estimate using 1970s prices
# as well as the upper and the lower bounds to today's prices.

today_median_price = 583.3
scale_factor = today_median_price / np.median(boston_dataset.target)

The estimated property value is USD 827000.0.
At 65% confidence the valuation range is
USD 685000.0 at lower end to USD 685000.0 at the high end


In [85]:
def get_dollar_estimate(rm, ptratio, chas=False, large_Range=True):
    """Estimate the price of a property in Boston
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school in the area
    char -- True if the property is next to the river, False otherwise
    large_range -- True for a 95% prediction interval, False for a 68% interval
    
    """
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, ptratio,
                                              next_to_river=chas, high_confidence=large_Range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * scale_factor
    dollar_hi = np.e**upper * 1000 * scale_factor
    dollar_low = np.e**lower * 1000 * scale_factor

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is USD {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at lower end to USD {rounded_hi} at the high end')

In [86]:
get_dollar_estimate(rm=1, ptratio=30, chas=True)

The estimated property value is USD 254000.0.
At 95% confidence the valuation range is
USD 174000.0 at lower end to USD 369000.0 at the high end


In [87]:
import boston_valuation as val

In [88]:
val.get_dollar_estimate(rm=1, ptratio=30, chas=True)

The estimated property value is USD 254000.0.
At 95% confidence the valuation range is
USD 174000.0 at lower end to USD 369000.0 at the high end
